In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import os
import timeit

#날짜를 정해서 가져오고 model을 통해 evalute, prediction
#we have to predict overtime sequence .
# Make preprocessed datasets for test.
directory = "stock_data" 
fns = sorted( os.listdir(directory) )
ticker= "AAPL"
#ticker_data = "AAPL" +".csv"
#directory = directory + ticker_data

df = pd.DataFrame()

for fn in fns:
    if fn.find(ticker ) != -1 : # find the data of ficker :
        df_tmp = pd.read_csv( os.path.join(directory,fn) )
        df = pd.concat([df, df_tmp])
time_length = 78

In [22]:
#날짜를 정해서 가져오고 model을 통해 evalute, prediction
#we have to predict overtime sequence .
# Make preprocessed datasets for test.
directory = "stock_data_one_min" 
fns = sorted( os.listdir(directory) )
ticker= "AAPL"
#ticker_data = "AAPL" +".csv"
#directory = directory + ticker_data

df = pd.DataFrame()

for fn in fns:
    if fn.find(ticker ) != -1 :
    #if fn.find(ticker ) != -1 and (fn.find('2015')==0 or fn.find('2016')==0 or fn.find('2017')==0 or\
    #                               fn.find('2018')==0 or fn.find('2019')==0 or fn.find('2020')==0 or fn.find('2021')==0)  : # find the data of ficker :
        df_tmp = pd.read_csv( os.path.join(directory,fn) )
        df = pd.concat([df, df_tmp])
time_length = 78 *5 

In [23]:
print(df.shape)
df = df.groupby('day').filter(lambda x: len(x)==time_length)  
print(df.shape)
num_days = df.shape[0] / time_length
num_days = (int) (num_days) 

print(df.head())

(1075300, 9)
(1052610, 9)
          day   one_min  min_price  max_price  weighted_avg   avg_price  \
0  2011-01-03  09:30:00     325.00     326.00    325.464146  325.433034   
1  2011-01-03  09:31:00     324.84     325.69    325.312515  325.312195   
2  2011-01-03  09:32:00     325.39     325.62    325.470915  325.471015   
3  2011-01-03  09:33:00     325.19     325.59    325.422060  325.413896   
4  2011-01-03  09:34:00     325.24     325.90    325.441113  325.347761   

   volume  first_price  last_price  
0   84050       325.65      325.26  
1   56274       325.29      325.59  
2   25533       325.62      325.44  
3   32155       325.43      325.35  
4   24416       325.35      325.42  


In [24]:
df['index'] = np.arange(df.shape[0])
df.set_index(df['index'], inplace=True)
df.drop(['index', 'avg_price'] ,axis=1, inplace=True)

In [25]:
df.columns

Index(['day', 'one_min', 'min_price', 'max_price', 'weighted_avg', 'volume',
       'first_price', 'last_price'],
      dtype='object')

In [26]:
# find the idx from day
split_day = ['2014-06-09','2020-08-31']
split_ratio = [7, 4]
split_col_str = []
split_col_num = []
for col in df.columns :
    if type(df[col][0]) == str :
        split_col_str.append(col)
    else :
        split_col_num.append(col)
        
split_idx = []
for split_day_idx in split_day :
    split_idx.append(df[df['day'] == split_day_idx].index[0])

In [27]:
# adjust the price and volume from the day_idx and ratio
for split,ratio in zip(split_idx, split_ratio) :
    df_drop = df[:split].copy()
    df_drop.drop(split_col_str, axis=1, inplace=True)

    df_drop['min_price'] = df_drop['min_price'] / ratio
    df_drop['max_price'] = df_drop['max_price'] / ratio
    df_drop['weighted_avg'] = df_drop['weighted_avg'] / ratio
    df_drop['first_price'] = df_drop['first_price'] / ratio
    df_drop['last_price'] = df_drop['last_price'] / ratio
    df_drop['volume'] = df_drop['volume'] * ratio
    
    print(df_drop)
    for col in split_col_num:
        df.iloc[:split, df.columns.get_loc(col)] = df_drop[col]


        min_price  max_price  weighted_avg  volume  first_price  last_price
index                                                                      
0       46.428571  46.571429     46.494878  588350    46.521429   46.465714
1       46.405714  46.527143     46.473216  393918    46.470000   46.512857
2       46.484286  46.517143     46.495845  178731    46.517143   46.491429
3       46.455714  46.512857     46.488866  225085    46.490000   46.478571
4       46.462857  46.557143     46.491588  170912    46.478571   46.488571
...           ...        ...           ...     ...          ...         ...
318235  92.222857  92.260000     92.239427  295841    92.231429   92.228571
318236  92.214286  92.241429     92.229066  210028    92.231429   92.222857
318237  92.174286  92.231429     92.205175  369033    92.221400   92.207143
318238  92.182857  92.225714     92.208943  549822    92.207143   92.212843
318239  92.185714  92.240000     92.214635  990885    92.211429   92.224286

[318240 row

In [30]:
df

,day,one_min,min_price,max_price,weighted_avg,volume,first_price,last_price
index,,,,,,,,
0,2011-01-03,09:30:00,11.607143,11.642857,11.623719,2353400,11.630357,11.616429
1,2011-01-03,09:31:00,11.601429,11.631786,11.618304,1575672,11.617500,11.628214
2,2011-01-03,09:32:00,11.621071,11.629286,11.623961,714924,11.629286,11.622857
3,2011-01-03,09:33:00,11.613929,11.628214,11.622216,900340,11.622500,11.619643
4,2011-01-03,09:34:00,11.615714,11.639286,11.622897,683648,11.619643,11.622143
...,...,...,...,...,...,...,...,...
1052605,2021-12-31,15:55:00,177.510000,177.640000,177.572309,457589,177.590000,177.529400
1052606,2021-12-31,15:56:00,177.460000,177.680000,177.548542,595430,177.529400,177.500000
1052607,2021-12-31,15:57:00,177.480000,177.595300,177.528050,419235,177.500000,177.560000


In [29]:
df.to_csv('./stock_data_prepro/AAPL_min.csv', index=False)